In [1]:
import os
import chardet
import numpy as np
import pandas as pd
import joblib

In [2]:
url = 'https://bit.ly/Seafood_Trade_2021yr'

In [3]:
df = pd.read_csv(url)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31865 entries, 0 to 31864
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ymonth  31865 non-null  int64  
 1   nation  31865 non-null  object 
 2   itemNm  31865 non-null  object 
 3   imxprt  31865 non-null  object 
 4   weight  31865 non-null  float64
 5   dollar  31865 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 1.5+ MB


In [5]:
df.head()

,ymonth,nation,itemNm,imxprt,weight,dollar
0,202101,가나,고등어,수출,107300.000,121443
1,202101,가나,기타,수입,10.000,2808
2,202101,가나,다랑어,수입,47446.800,241880
3,202101,가나,전갱이,수출,108000.000,106400
4,202101,가봉,어류,수입,1.615,26


In [6]:
df['ymonth'] = df['ymonth'].astype(str)

In [7]:
df.dtypes

ymonth     object
nation     object
itemNm     object
imxprt     object
weight    float64
dollar      int64
dtype: object

In [8]:
df['dollar'] = df['dollar'] / 1000

In [9]:
df.head()

,ymonth,nation,itemNm,imxprt,weight,dollar
0,202101,가나,고등어,수출,107300.000,121.443
1,202101,가나,기타,수입,10.000,2.808
2,202101,가나,다랑어,수입,47446.800,241.880
3,202101,가나,전갱이,수출,108000.000,106.400
4,202101,가봉,어류,수입,1.615,0.026


In [14]:
elong = df.groupby(by = ['nation', 'imxprt'])[['dollar']].sum().reset_index()
elong

,nation,imxprt,dollar
0,가나,수입,1847.987
1,가나,수출,8088.674
2,가봉,수입,0.026
3,가이아나,수입,17.290
4,가이아나,수출,0.036
...,...,...,...
274,필리핀,수출,37663.369
275,헝가리,수입,1.957
276,헝가리,수출,18.857
277,홍콩,수입,32389.424


In [16]:
widen = elong.pivot(index = 'nation', columns = 'imxprt', values = 'dollar')
widen

imxprt,수입,수출
nation,,
가나,1847.987,8088.674
가봉,0.026,NaN
가이아나,17.290,0.036
감비아,2065.081,NaN
과테말라,NaN,131.373
...,...,...
피지,1.362,69.910
핀란드,5.993,377.437
필리핀,29771.506,37663.369


In [19]:
widen = widen.fillna(value = 0)
widen

imxprt,수입,수출
nation,,
가나,1847.987,8088.674
가봉,0.026,0.000
가이아나,17.290,0.036
감비아,2065.081,0.000
과테말라,0.000,131.373
...,...,...
피지,1.362,69.910
핀란드,5.993,377.437
필리핀,29771.506,37663.369


In [21]:
widen = widen.reset_index()

In [22]:
widen.columns

Index(['nation', '수입', '수출'], dtype='object', name='imxprt')

In [23]:
widen.columns.name

'imxprt'

In [24]:
widen.columns.name = None

In [25]:
widen

,nation,수입,수출
0,가나,1847.987,8088.674
1,가봉,0.026,0.000
2,가이아나,17.290,0.036
3,감비아,2065.081,0.000
4,과테말라,0.000,131.373
...,...,...,...
160,피지,1.362,69.910
161,핀란드,5.993,377.437
162,필리핀,29771.506,37663.369
163,헝가리,1.957,18.857


In [26]:
widen = widen.rename(columns = {'nation': '국가'})
widen

,국가,수입,수출
0,가나,1847.987,8088.674
1,가봉,0.026,0.000
2,가이아나,17.290,0.036
3,감비아,2065.081,0.000
4,과테말라,0.000,131.373
...,...,...,...
160,피지,1.362,69.910
161,핀란드,5.993,377.437
162,필리핀,29771.506,37663.369
163,헝가리,1.957,18.857


In [27]:
widen = widen[['국가', '수출', '수입']]
widen

,국가,수출,수입
0,가나,8088.674,1847.987
1,가봉,0.000,0.026
2,가이아나,0.036,17.290
3,감비아,0.000,2065.081
4,과테말라,131.373,0.000
...,...,...,...
160,피지,69.910,1.362
161,핀란드,377.437,5.993
162,필리핀,37663.369,29771.506
163,헝가리,18.857,1.957


In [28]:
widen['무역수지'] = widen['수출'] - widen['수입']

C:\Users\15U40N\AppData\Local\Temp\ipykernel_9112\4128859706.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  widen['무역수지'] = widen['수출'] - widen['수입']


In [29]:
widen

,국가,수출,수입,무역수지
0,가나,8088.674,1847.987,6240.687
1,가봉,0.000,0.026,-0.026
2,가이아나,0.036,17.290,-17.254
3,감비아,0.000,2065.081,-2065.081
4,과테말라,131.373,0.000,131.373
...,...,...,...,...
160,피지,69.910,1.362,68.548
161,핀란드,377.437,5.993,371.444
162,필리핀,37663.369,29771.506,7891.863
163,헝가리,18.857,1.957,16.900


In [30]:
widen = widen.sort_values(by = ['무역수지'])

In [31]:
widen.head(n = 10)

,국가,수출,수입,무역수지
29,러시아,54078.994,1219825.619,-1165746.625
58,베트남,177741.726,808388.014,-630646.288
17,노르웨이,1884.646,575973.581,-574088.935
119,중국,726529.129,1300449.248,-573920.119
153,페루,2710.209,165384.173,-162673.964
122,칠레,1426.241,119811.899,-118385.658
114,인도,687.022,106905.400,-106218.378
127,캐나다,41938.119,113857.715,-71919.596
89,아르헨티나,53.530,69720.069,-69666.539
22,대만,60066.032,129664.240,-69598.208


In [32]:
widen.tail(n = 10)

,국가,수출,수입,무역수지
50,미령사모아,8649.957,811.309,7838.648
162,필리핀,37663.369,29771.506,7891.863
14,네덜란드,20403.436,5373.894,15029.542
87,싱가포르,21449.930,2593.154,18856.776
164,홍콩,54560.233,32389.424,22170.809
25,독일,27186.712,1263.392,25923.320
113,이탈리아,52961.214,25683.273,27277.941
158,프랑스,66501.818,34315.325,32186.493
49,미국,395386.470,250113.606,145272.864
116,일본,635499.670,170243.592,465256.078


In [33]:
widen['수지구분'] = np.where(widen['무역수지'].ge(0), '흑자', '적자')
widen.head()

,국가,수출,수입,무역수지,수지구분
29,러시아,54078.994,1219825.619,-1165746.625,적자
58,베트남,177741.726,808388.014,-630646.288,적자
17,노르웨이,1884.646,575973.581,-574088.935,적자
119,중국,726529.129,1300449.248,-573920.119,적자
153,페루,2710.209,165384.173,-162673.964,적자


In [34]:
widen['수지구분'].value_counts()

흑자    84
적자    81
Name: 수지구분, dtype: int64

In [35]:
widen['무역수지'].mean()

-20329.818072727277

In [36]:
widen['무역수지'].median()

0.038

In [37]:
widen['수지구분2'] = np.select(
    condlist = [
        widen['무역수지'].ge(10000), 
        widen['무역수지'].ge(0),
        widen['무역수지'].ge(-10000),
        widen['무역수지'].lt(-10000),
    ],
    choicelist = ['흑자1', '흑자2', '적자1', '적자2']
)

In [38]:
widen.head()

,국가,수출,수입,무역수지,수지구분,수지구분2
29,러시아,54078.994,1219825.619,-1165746.625,적자,적자2
58,베트남,177741.726,808388.014,-630646.288,적자,적자2
17,노르웨이,1884.646,575973.581,-574088.935,적자,적자2
119,중국,726529.129,1300449.248,-573920.119,적자,적자2
153,페루,2710.209,165384.173,-162673.964,적자,적자2


In [40]:
widen['수지구분2'].value_counts(normalize = True).sort_index()

적자1    0.327273
적자2    0.163636
흑자1    0.048485
흑자2    0.460606
Name: 수지구분2, dtype: float64

In [44]:
df1 = df[df['nation'].eq('일본')]
pvt = pd.pivot_table(data = df1, 
                     index = 'itemNm', 
                     columns = 'imxprt', 
                     values = 'dollar', 
                     aggfunc = np.sum) \
  .reset_index() \
  .fillna(value = 0)
pvt

imxprt,itemNm,수입,수출
0,가공하지 않은 것,0.350,0.041
1,가공한 것,6.018,1.059
2,가다랑어,4.409,2580.066
3,가리비과,39305.985,13.476
4,가오리,4622.530,0.000
...,...,...,...
156,해파리,382.517,0.000
157,홍어,1010.925,0.000
158,홍합,0.016,104.642
159,황다랑어,0.000,15265.701


In [45]:
pvt['무역수지'] = pvt['수출'] - pvt['수입']
pvt = pvt.sort_values(by = ['무역수지'])
pvt

imxprt,itemNm,수입,수출,무역수지
3,가리비과,39305.985,13.476,-39292.509
50,돔,36402.686,21.139,-36381.547
66,방어,36058.005,3505.671,-32552.334
117,우렁쉥이,7345.288,0.000,-7345.288
59,명태,6158.574,100.207,-6058.367
...,...,...,...,...
33,냉동한 것,501.435,22728.785,22227.350
77,붕장어,0.000,31125.801,31125.801
124,전복,36.525,42698.612,42662.087
26,김,436.750,114139.670,113702.920
